In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import csv

# 初始化 WebDriver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# 定義目標 URL
url = "https://maps.app.goo.gl/VyfjQnfxF6KVyVnK8"
driver.get(url)
time.sleep(2)

# 抓取景點名稱
try:
    title_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf"))
    )
    title = title_element.text
    print(f"景點名稱: {title}")
except Exception as e:
    print("無法抓取景點名稱:", e)
    driver.quit()  # 直接退出瀏覽器
    exit()  # 確保程式結束執行

# 確保評論區按鈕生成
scroll_attempts = 0
button_found = False

while scroll_attempts < 5 and not button_found:
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(random.uniform(1, 2))
    try:
        comment_button = driver.find_element(By.CSS_SELECTOR, "button.HHrUdb")
        button_found = True
    except Exception:
        scroll_attempts += 1

if not button_found:
    print("無法找到評論區按鈕，請檢查網頁結構或滾動邏輯。")
    driver.quit()

# 點擊評論區按鈕
try:
    comment_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.HHrUdb"))
    )
    comment_button.click()
    time.sleep(random.uniform(1, 2))
except Exception as e:
    print("無法找到評論區按鈕:", e)
    driver.quit()

# 更改排序為"最新"
try:
    sort_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-value=\'排序\']"))
    )
    sort_button.click()
    time.sleep(1)

    latest_button = driver.find_elements(By.CSS_SELECTOR, "div.mLuXec")[1]
    driver.execute_script("arguments[0].scrollIntoView();", latest_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", latest_button)
    time.sleep(2)
except Exception as e:
    print("無法更改排序為最新:", e)
    driver.quit()

# 滑動頁面並抓取評論
def parse_date(date_str):
    return date_str  # 保留原始日期文本

comments = []
loaded_reviews = 0
unique_review_ids = set()  # 用於檢查重複的評論 ID

repeat_attempts = 0
max_repeat_attempts = 10  # 若連續5次抓取到重複評論則結束

while repeat_attempts < max_repeat_attempts:
    reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")

    for review in reviews:
        try:
            review_id = review.get_attribute("data-review-id")
            if review_id in unique_review_ids:
                continue  # 跳過重複的評論

            unique_review_ids.add(review_id)

            rating_element = review.find_element(By.CSS_SELECTOR, "span.kvMYJc")
            rating = rating_element.get_attribute("aria-label").replace(" 顆星", "")

            # 處理評論內容
            if review.find_elements(By.CSS_SELECTOR, "button.kyuRq.WOKzJe"):
                review.find_element(By.CSS_SELECTOR, "button.kyuRq.WOKzJe").click()
                time.sleep(1)

            if review.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq"):
                review.find_element(By.CSS_SELECTOR, "button.w8nwRe.kyuRq").click()
                time.sleep(1)

            comment_elements = review.find_elements(By.CSS_SELECTOR, "span.wiI7pd")
            comment_content = comment_elements[0].text if comment_elements else ""

            date_element = review.find_element(By.CSS_SELECTOR, "span.rsqaWe")
            date_time = parse_date(date_element.text)

            comments.append({
                "ID": review_id,
                "Rating": rating,
                "Comment": comment_content,
                "Date": date_time
            })
            loaded_reviews += 1
            print(f"正在處理第{loaded_reviews}則評論")
        except Exception as e:
            print("評論抓取錯誤:", e)

    # 檢查評論是否新增
    if len(comments) == loaded_reviews:
        repeat_attempts += 1
        print(f"評論未新增，第 {repeat_attempts} 次嘗試。")
    else:
        repeat_attempts = 0

    # 滑動頁面加載更多評論
    driver.execute_script("arguments[0].scrollIntoView();", reviews[-1])
    time.sleep(random.uniform(1.5, 2.5))

print("已連續多次抓取到重複評論，停止抓取。")

# 等待最後的評論加載完成
time.sleep(5)

# 保存評論到 CSV
csv_file = "comments_google_map.csv"
with open(csv_file, "w", newline="", encoding="utf-8-sig") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Rating", "Comment", "Date"])
    writer.writeheader()
    writer.writerows(comments)

print(f"成功抓取 {len(comments)} 則評論並保存至 {csv_file}")

driver.quit()
